In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import numpy as np
import pandas
import os
from tqdm import tqdm
import random

from graph import Graph
from knowledgegraph import KnowledgeGraph
from rule import Rule
from train import train
from train import train_step_by_step
from train import train_pow_decay
from device import device
import pickle

# data_path = 'data/wn18rr/train.txt'
data_path = 'data/umls/train.txt'
# data_path = 'temp.txt'
# data_path = 'data/data_2_rule.txt'
# data_path = 'data/family/train.txt'
# data_path = 'data/noisy.txt'
# data_path = 'data/kinship/train.txt'
KG = KnowledgeGraph(data_path)
# KG.graph.tosparse()

EPOCH = 100
# BATCH_SIZE = 294
BATCH_SIZE = 1000
rule_body_len = 3
# target_predicate = KG.predicate2id[1]
target_predicate = 0
# target_predicate = KG.predicate2id['affects']

# target_triplets = list()
# for triplet in KG.triplets:
#     if (triplet[1] == target_predicate):
#         target_triplets.append(triplet)
# n_triplets = len(target_triplets)
# v_froms = set()
# for triplet in target_triplets:
#     v_froms.add(triplet[0])
# print(len(target_triplets))

# model = train_step_by_step(rule_body_len, target_predicate, KG, BATCH_SIZE, EPOCH, lr=1e-2)
# model0 = train_pow_decay(rule_body_len, target_predicate, KG, BATCH_SIZE, EPOCH, lr=1e-5, decay_st=100)

# KG.print_model(model0, KG.id2predicate[target_predicate])

c:\Users\tourx\source\repos\sota\knowledgegraph.py:13: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_entity = data[0].append(data[2])


In [2]:
with open("generate.txt", "w") as f:
        f.write("[")
accs = list()

pos_triplets = KG.graph.A.clone().detach()
neg_triplets = (1.0 - KG.graph.A).detach()

with tqdm(range(KG.graph.n_edge_type), ncols=80) as _tqdm:
    for target_predicate in _tqdm:
        all_triplets = list()
        offset = KG.predicate_offset
        for triplet in KG.triplets:
            if (triplet[1] == target_predicate):
                all_triplets.append(triplet)
            if (triplet[1] == target_predicate + offset):
                all_triplets.append(triplet)
            if (triplet[1] == target_predicate - offset):
                all_triplets.append(triplet)
        if (target_predicate >= KG.n_predicate_original()):
            offset = -offset
        # for triplet in all_triplets:
        #     KG.graph.mask_edge(triplet[0], triplet[2], triplet[1])
        N_RULE = 100

        with open("generate.txt", "a+") as f:
            f.write("[")
        
        loop = 0
        randinit = False
        npos = torch.sum(pos_triplets[target_predicate])
        nneg = torch.sum(neg_triplets[target_predicate])
        while loop < N_RULE:
            model = train(rule_body_len, target_predicate, KG, BATCH_SIZE, pos_triplets[target_predicate], neg_triplets[target_predicate],
                EPOCH=10, lr=1e-2, decay_st=100, randinit=True)
            randinit=False
            # KG.print_model(model, KG.id2predicate[target_predicate])

            rule = KG.output_model(model, KG.id2predicate[target_predicate])
            modelpre = KG.construct_model(rule)

            total = 0
            cnt = 0

            mask0 = torch.ones((1, KG.graph.n_vertex)).to(device)
            a = (modelpre(KG.graph) + 0.3).type(torch.long).type(torch.float32).T
            for v_from in range(KG.graph.n_vertex):
                mask1 = (KG.graph.A[KG.predicate2id[rule[0]]].T[v_from]+0.3).type(torch.long).type(torch.float32)
                ans = a[v_from]
                total += int(mask0.matmul(ans.T))
                cnt += int(mask1.matmul(ans.T))

            num = int(torch.sum(KG.graph.A[KG.predicate2id[rule[0]]]))

            # if (cnt == num):
            #     randinit=True
            loop += 1
            if (total == 0):
                randinit=True
                continue
            acc = cnt / total
            accs.append(acc)
            
            a = (modelpre(KG.graph) + 0.3).type(torch.long)
            ratio = 1.0 - acc * a
            pos_triplets[target_predicate] *= ratio
            neg_triplets[target_predicate] /= (ratio + 1e-10)
            pos_triplets[target_predicate] *= npos / (torch.sum(pos_triplets[target_predicate]) + 1e-10)
            neg_triplets[target_predicate] *= nneg / (torch.sum(neg_triplets[target_predicate]) + 1e-10)


            with open("generate.txt", "a+") as f:
                f.write(str(rule) + ",\n")
        with open("generate.txt", "a+") as f:
            f.write("],")
        # KG.graph.restore_masked_edge()
with open("generate.txt", "a+") as f:
        f.write("]")

100%|███████████████████████████████████████████| 90/90 [19:55<00:00, 13.29s/it]


In [23]:
sum = 0.0
for i in accs:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
    sum += i
sum / len(accs)

0.22047989331743906

In [23]:
target_predicate= KG.predicate2id["affects"]
pos_triplets = KG.graph.A.clone().detach()
neg_triplets = (1.0 - KG.graph.A).detach()

model = train(rule_body_len, target_predicate, KG, BATCH_SIZE,pos_triplets, neg_triplets, EPOCH=100, lr=3e-3, decay_st=50, randinit=False)
randinit=False
KG.print_model(model, KG.id2predicate[target_predicate])

rule = KG.output_model(model, KG.id2predicate[target_predicate])
num, total, cnt = KG.check_rule(rule)
acc = cnt / total
if (cnt == num):
    randinit=True
if (total == 0):
    randinit=True
# print(acc)
modelpre = KG.construct_model(rule)
a = (modelpre(KG.graph) + 0.3).type(torch.long)
ratio = 1.0 - acc * a

Rule: 
[rev-degree_of, degree_of, rev-degree_of, ]
Score: 
[0.5500677227973938, 0.8389495611190796, 0.8700454235076904]
#True: 294
#Predicted True: 26
#True And Predicted True: 5

#True Among Predicted True: 0.19230769230769232

#Predicted True Among True: 0.017006802721088437


Parameter Matrix: 
tensor([[1.2115e-03, 9.1785e-04, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 8.8122e-03],
        [0.0000e+00, 7.0988e-08, 1.4359e-16],
        [0.0000e+00, 0.0000e+00, 0.0000e+00],
        [6.6354e-08, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 2.2744e-08, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.1872e-16, 0.0000e+00, 0.0000e+00],
        [1.2083e-02, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 1.3208e-16],
        [0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 4.0751e-08],
        [3.

In [9]:
model = train_pow_decay(rule_body_len, KG.graph.n_edge_type - 1, KG, BATCH_SIZE, EPOCH=50, lr=1e-4, decay_st=40, randinit=False)
KG.print_model(model, "rev-term19")

Rule: 
[rev-term1, rev-term18, rev-term4, ]
Score: 
[0.8116318583488464, 0.5812168717384338, 0.5183747410774231]
#True: 1
#Predicted True: 210
#True And Predicted True: 1

#True Among Predicted True: 0.004761904761904762

#Predicted True Among True: 1.0


Parameter Matrix: 
tensor([[3.7808e-09, 2.9658e-16, 3.6560e-17],
        [2.4195e-15, 3.2533e-16, 4.9660e-12],
        [1.4149e-10, 2.0584e-16, 9.4983e-11],
        [1.5064e-15, 6.9812e-16, 4.4324e-17],
        [8.0623e-12, 1.9110e-16, 3.4722e-17],
        [1.3863e-14, 2.1970e-04, 7.3875e-17],
        [8.2326e-14, 2.7482e-16, 4.8163e-01],
        [4.9409e-16, 2.4695e-12, 2.5858e-12],
        [1.0073e-15, 6.5767e-12, 4.2890e-17],
        [8.5555e-16, 4.1400e-16, 4.4625e-17],
        [2.2932e-12, 4.0365e-01, 3.6128e-17],
        [2.1377e-15, 4.8431e-09, 4.5572e-17],
        [2.5730e-11, 9.0266e-16, 6.0646e-12],
        [5.0679e-12, 3.4864e-15, 1.8889e-12],
        [9.4372e-16, 1.8130e-12, 3.9173e-17],
        [1.7382e-11, 4.7065e-11, 6.

In [ ]:
KG.predicate2id["carries_out"]

In [ ]:
torch.sort(torch.softmax(model.R[0].R, dim=0), descending=True)

In [5]:
rule = ["affects", "manifestation_of", "rev-isa", "rev-conceptually_related_to"]
KG.print_rule(rule)

#True: 494
#Predicted True: 7
#True And Predicted True: 1

#True Among Predicted True: 0.14285714285714285

#Predicted True Among True: 0.0020242914979757085



In [ ]:
rule = ["affects", "affects", "complicates", "degree_of"]
model2 = Rule(KG.n_predicate_total(), len(rule) - 1)
for i in range(model2.n_step):
    model2.R[i].R.requires_grad_(False)
for i in range(model2.n_step):
    for j in range(model2.n_predicate):
        model2.R[i].R[j] = float('-inf')
for i in range(0, len(rule) - 1):
    model2.R[i].R[KG.predicate2id[rule[i + 1]]] = 1.0

In [ ]:
# model2 = train_step_by_step(rule_body_len, target_predicate, KG, BATCH_SIZE, EPOCH, lr=1e-1, model=model2)
# KG.print_model(model2, KG.id2predicate[target_predicate])

In [ ]:
rec = 0
for i in target_triplets:
    ans = torch.max(model(i[0], KG.graph))
    rec = max(ans, rec)
rec

In [ ]:
with open("model", "wb") as f:
		pickle.dump(model, f)

In [ ]:
KG.print_rule(['affects', 'affects', 'isa', 'rev-co-occurs_with'])